## Data Loading and Summary

In [ ]:
!wget https://datasets.imdbws.com/title.basics.tsv.gz
!wget https://datasets.imdbws.com/title.episode.tsv.gz
!wget https://datasets.imdbws.com/title.ratings.tsv.gz

--2025-01-20 08:40:09--  https://datasets.imdbws.com/title.basics.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 3.167.212.123, 3.167.212.55, 3.167.212.77, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|3.167.212.123|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 200509675 (191M) [binary/octet-stream]
Saving to: ‘title.basics.tsv.gz’

title.basics.tsv.gz 100%[===================>] 191.22M   126MB/s    in 1.5s    

2025-01-20 08:40:10 (126 MB/s) - ‘title.basics.tsv.gz’ saved [200509675/200509675]

--2025-01-20 08:40:11--  https://datasets.imdbws.com/title.episode.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 3.167.212.123, 3.167.212.55, 3.167.212.77, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|3.167.212.123|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 48224273 (46M) [binary/octet-stream]
Saving to: ‘title.episode.tsv.gz’

title.episode.tsv.g 100%[===================>]  45.

In [ ]:
!gzip -d title.basics.tsv.gz
!gzip -d title.episode.tsv.gz
!gzip -d title.ratings.tsv.gz

## Data Overview

In [ ]:
import pandas as pd

In [ ]:
basics = pd.read_csv('title.basics.tsv', sep='\t', low_memory=False, na_values=['\\N'])
episode = pd.read_csv('title.episode.tsv', sep='\t', low_memory=False, na_values=['\\N'])
ratings = pd.read_csv('title.ratings.tsv', sep='\t', low_memory=False, na_values=['\\N'])

In [ ]:
basics.head(3)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0.0,1894.0,NaN,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0.0,1892.0,NaN,5,"Animation,Short"
2,tt0000003,short,Poor Pierrot,Pauvre Pierrot,0.0,1892.0,NaN,5,"Animation,Comedy,Romance"


In [ ]:
episode.head(3)

,tconst,parentTconst,seasonNumber,episodeNumber
0,tt0031458,tt32857063,NaN,NaN
1,tt0041951,tt0041038,1.0,9.0
2,tt0042816,tt0989125,1.0,17.0


In [ ]:
ratings.head(3)

,tconst,averageRating,numVotes
0,tt0000001,5.7,2117
1,tt0000002,5.6,285
2,tt0000003,6.4,2157


In [ ]:
len(episode), len(basics), len(ratings)

(8754842, 11389318, 1524613)

### Checking ID columns for uniqueness

In [ ]:
basics['tconst'].is_unique, episode['tconst'].is_unique, ratings['tconst'].is_unique

(True, True, True)

In [ ]:
# Check if columns 'tConst' and 'parentTconst' in episode have common values
len(set(episode['tconst']).intersection(set(episode['parentTconst'])))

0

## Data Cleaning

### Data Cleaning for `basics` Dataset

In [ ]:
basics.dtypes

,0
tconst,object
titleType,object
primaryTitle,object
originalTitle,object
isAdult,float64
startYear,float64
endYear,float64
runtimeMinutes,object
genres,object


In [ ]:
# Checking for null values
basics.isna().sum()

,0
tconst,0
titleType,0
primaryTitle,19
originalTitle,19
isAdult,1
startYear,1421908
endYear,11254612
runtimeMinutes,7777889
genres,503208


In [ ]:
# view all rows with null values of 'primaryTitle' or originalTitle'
basics[(basics['primaryTitle'].isna()) | (basics['originalTitle'].isna())]

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
1253193,tt10516578,video,NaN,NaN,0.0,2017.0,NaN,NaN,"Music,Short"
3428270,tt14510930,tvEpisode,NaN,NaN,0.0,NaN,NaN,NaN,NaN
4070202,tt15700278,tvEpisode,NaN,NaN,0.0,2021.0,NaN,NaN,Talk-Show
4550901,tt17042812,movie,NaN,NaN,0.0,2010.0,NaN,87,Thriller
5058312,tt1971246,tvEpisode,NaN,NaN,0.0,2011.0,NaN,NaN,Biography
5253934,tt2067043,tvEpisode,NaN,NaN,0.0,1965.0,NaN,NaN,Music
5592129,tt21883066,tvEpisode,NaN,NaN,0.0,2022.0,NaN,NaN,"News,Talk-Show"
5861484,tt2305914,tvEpisode,NaN,NaN,0.0,NaN,NaN,NaN,"Comedy,Talk-Show"
5861491,tt2305918,tvEpisode,NaN,NaN,0.0,NaN,NaN,NaN,"Comedy,Talk-Show"
7529083,tt31462159,tvEpisode,NaN,NaN,0.0,2024.0,NaN,NaN,Drama


17 rows have missing values for both `primaryTitle` and `originalTitle` fields. We will remove these rows.

In [ ]:
# Remove rows with null values in 'primaryTitle' and 'originalTitle'
basics = basics.dropna(subset=['primaryTitle', 'originalTitle'])

In [ ]:
# View the only row with missing 'isAdult' value
pd.set_option('max_colwidth', None)
basics[basics['isAdult'].isna()]

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
2987657,tt13704268,tvEpisode,Bay of the Triffids/Doctor of Doom\tBay of the Triffids/Doctor of Doom,0,NaN,NaN,NaN,"Animation,Comedy,Family",NaN


In this case, the `\t` character was intepreted as literal string isntead of a `Tab` seperator.

Instead of removing this row, we fix it by reassigning the `primaryTitle`, `originalTitle`, and `isAdult` fields to their right values:
- `primaryTitle`: Bay of the Triffids/Doctor of Doom
- `originalTitle`: Bay of the Triffids/Doctor of Doom
- isAdult: 0

In [ ]:
basics.loc[2989193, ['primaryTitle', 'originalTitle', 'isAdult']] = ['Bay of the Triffids/Doctor of Doom', 'Bay of the Triffids/Doctor of Doom', 0]
basics.loc[2989193]

,2989193
tconst,tt13707222
titleType,short
primaryTitle,Bay of the Triffids/Doctor of Doom
originalTitle,Bay of the Triffids/Doctor of Doom
isAdult,0.0
startYear,2020.0
endYear,NaN
runtimeMinutes,20
genres,"Sci-Fi,Short"


Next, we check if all values in the field `isAdult` is either 0 or 1

In [ ]:
basics['isAdult'].value_counts()

,count
isAdult,
0.0,11020436
1.0,368042
1978.0,130
1985.0,83
1980.0,66
1979.0,63
1984.0,41
1974.0,32
1982.0,32


It seems like the some of the values in the field `isAdult` are year values and should belong to the `startYear` columns instead.

We closely inspect these rows

In [ ]:
basics_isAdult_filter = basics[~basics['isAdult'].isin([0,1])]
basics_isAdult_filter

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
1096717,tt10233364,tvEpisode,Rolling in the Deep Dish\tRolling in the Deep Dish,0,2019.0,NaN,NaN,Reality-TV,NaN
1506173,tt10970874,tvEpisode,Die Bauhaus-Stadt Tel Aviv - Vorbild für die Metropolen der Moderne?\tDie Bauhaus-Stadt Tel Aviv - Vorbild für die Metropolen der Moderne?,0,2019.0,NaN,NaN,Talk-Show,NaN
1893321,tt11670006,tvEpisode,...ein angenehmer Unbequemer...\t...ein angenehmer Unbequemer...,0,1981.0,NaN,NaN,Documentary,NaN
2004106,tt11868642,tvEpisode,GGN Heavyweight Championship Lungs With Mike Tyson and Snoop\tGGN Heavyweight Championship Lungs With Mike Tyson and Snoop,0,2020.0,NaN,NaN,Talk-Show,NaN
2157353,tt12149332,tvEpisode,Jeopardy! College Championship Semifinal Game 3\tJeopardy! College Championship Semifinal Game 3,0,2020.0,NaN,NaN,"Family,Game-Show",NaN
...,...,...,...,...,...,...,...,...,...
8525558,tt35473846,tvEpisode,X '1963\tX '1963,0,1985.0,NaN,NaN,"Action,Fantasy,Horror",NaN
8525574,tt35473863,tvEpisode,Reptilicus '1961\tReptilicus '1961,0,1985.0,NaN,NaN,"Action,Fantasy,Horror",NaN
8717074,tt3984412,tvEpisode,"I'm Not Going to Come Last, I'm Just Going to Die on The Amazing Race\tI'm Not Going to Come Last, I'm Just Going to Die on The Amazing Race",0,2014.0,NaN,NaN,"Game-Show,Reality-TV",NaN
11346028,tt9822816,tvEpisode,Zwischen Vertuschung und Aufklärung - Missbrauchsgipfel im Vatikan\tZwischen Vertuschung und Aufklärung - Missbrauchsgipfel im Vatikan,0,2019.0,NaN,NaN,Talk-Show,NaN


Looks like this is the same issue as before, where the `\t` seperators were intepreted as literal string.

We will check if all these rows contain a `\t` character, and if every row with value of `isAdult` not equal to 0 or 1 have missing `startYear` values

In [ ]:
# Check if every row in basics_isAdult_filter contains `\t` in the primaryTitle
all(basics_isAdult_filter['primaryTitle'].str.contains('\t'))

True

In [ ]:
basics_isAdult_filter['originalTitle'].value_counts()

,count
originalTitle,
0,819
1,2


In [ ]:
basics_isAdult_filter['startYear'].unique()

array([nan])

We conclude that for every row where `isAdult` is neither 0 nor 1:

- Every value in `primaryTitle` contains '\t', which should be the `Tab` separator
- Every value in `originalTitle` is either 0 or 1, which should belong in the `isAdult` column
- Every value in `startYear` is missing, which is currently in the `isAdult` column

We will fix these rows by:
- Splitting all titles in `primaryTitle` by the `\t` character into `primaryTitle` and `originalTitle`
- Moving all current values in `originalTitle`, which are 0 or 1, to the `isAdult` field
- Moving all current values in `originalTitle` (e.g 1985, 1986) to the `startYear` field

In [ ]:
# pd.set_option('mode.chained_assignment', None)
basics_isAdult_filter['startYear'] = basics_isAdult_filter['isAdult']
basics_isAdult_filter['isAdult'] = basics_isAdult_filter['originalTitle']
basics_isAdult_filter[['primaryTitle', 'originalTitle']] = basics_isAdult_filter['primaryTitle'].str.split('\t', expand=True)
basics_isAdult_filter

<ipython-input-22-2e15d070df65>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  basics_isAdult_filter['startYear'] = basics_isAdult_filter['isAdult']
<ipython-input-22-2e15d070df65>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  basics_isAdult_filter['isAdult'] = basics_isAdult_filter['originalTitle']
<ipython-input-22-2e15d070df65>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
1096717,tt10233364,tvEpisode,Rolling in the Deep Dish,Rolling in the Deep Dish,0,2019.0,NaN,Reality-TV,NaN
1506173,tt10970874,tvEpisode,Die Bauhaus-Stadt Tel Aviv - Vorbild für die Metropolen der Moderne?,Die Bauhaus-Stadt Tel Aviv - Vorbild für die Metropolen der Moderne?,0,2019.0,NaN,Talk-Show,NaN
1893321,tt11670006,tvEpisode,...ein angenehmer Unbequemer...,...ein angenehmer Unbequemer...,0,1981.0,NaN,Documentary,NaN
2004106,tt11868642,tvEpisode,GGN Heavyweight Championship Lungs With Mike Tyson and Snoop,GGN Heavyweight Championship Lungs With Mike Tyson and Snoop,0,2020.0,NaN,Talk-Show,NaN
2157353,tt12149332,tvEpisode,Jeopardy! College Championship Semifinal Game 3,Jeopardy! College Championship Semifinal Game 3,0,2020.0,NaN,"Family,Game-Show",NaN
...,...,...,...,...,...,...,...,...,...
8525558,tt35473846,tvEpisode,X '1963,X '1963,0,1985.0,NaN,"Action,Fantasy,Horror",NaN
8525574,tt35473863,tvEpisode,Reptilicus '1961,Reptilicus '1961,0,1985.0,NaN,"Action,Fantasy,Horror",NaN
8717074,tt3984412,tvEpisode,"I'm Not Going to Come Last, I'm Just Going to Die on The Amazing Race","I'm Not Going to Come Last, I'm Just Going to Die on The Amazing Race",0,2014.0,NaN,"Game-Show,Reality-TV",NaN
11346028,tt9822816,tvEpisode,Zwischen Vertuschung und Aufklärung - Missbrauchsgipfel im Vatikan,Zwischen Vertuschung und Aufklärung - Missbrauchsgipfel im Vatikan,0,2019.0,NaN,Talk-Show,NaN


In [ ]:
# Replace the rows in the original DataFrame
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

basics.loc[basics_isAdult_filter.index] = basics_isAdult_filter
basics.head(3)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0.0,1894.0,NaN,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0.0,1892.0,NaN,5,"Animation,Short"
2,tt0000003,short,Poor Pierrot,Pauvre Pierrot,0.0,1892.0,NaN,5,"Animation,Comedy,Romance"


In [ ]:
basics['isAdult'] = pd.to_numeric(basics['isAdult'])

In [ ]:
basics.isna().sum()

,0
tconst,0
titleType,0
primaryTitle,0
originalTitle,0
isAdult,0
startYear,1421084
endYear,11254593
runtimeMinutes,7777874
genres,503207


There are still missing values in `startYear`, `endYear`, `runtimeMinute`, and `genres`. However, these fields are not mandatory and we can safely ignore these missing values.

Inspect rows with missing values and ensure they are valid

In [ ]:
basics[basics.startYear.isna()]

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
619,tt0000624,short,The Ugly Duckling,The Ugly Duckling,0.0,NaN,NaN,NaN,Short
65745,tt0067098,tvEpisode,Willi Forst,Willi Forst,0.0,NaN,NaN,55,NaN
83788,tt0085677,tvEpisode,High Country,High Country,0.0,NaN,NaN,NaN,Sport
90911,tt0092975,tvEpisode,Erste Liebe und Heiratssachen,Erste Liebe und Heiratssachen,0.0,NaN,NaN,NaN,Documentary
90957,tt0093025,movie,Tales of the Brothers Quay,The Films of the Brothers Quay,0.0,NaN,NaN,78,"Animation,Documentary"
...,...,...,...,...,...,...,...,...,...
11389206,tt9916616,short,Terror,Terror,0.0,NaN,NaN,13,"Drama,Short"
11389208,tt9916620,movie,The Copeland Case,The Copeland Case,0.0,NaN,NaN,NaN,Drama
11389222,tt9916652,short,Untitled well- being Documentary,Untitled well- being Documentary,0.0,NaN,NaN,NaN,Short
11389228,tt9916664,short,Untitled Land Army Girls Documentary,Untitled Land Army Girls Documentary,0.0,NaN,NaN,NaN,Short


Now that all values in `primaryTitle`, `originalTitle`, `isAdult` fields have been reassigned and moved to their corresponding field, we will filter out all adult movies, which have `isAdult` equal to 1

In [ ]:
# Remove all rows with 'isAdult' = 1
basics = basics[basics['isAdult'] == 0]
basics['isAdult'].value_counts()

,count
isAdult,
0.0,11021255


Finally, we reformat the `genres` column by adding a whitespace `\s` after every comma `,`

In [ ]:
basics['genres'] = basics['genres'].str.replace(r',', r', ')
basics['genres'].head(3)

,genres
0,"Documentary, Short"
1,"Animation, Short"
2,"Animation, Comedy, Romance"


Check all column types

In [ ]:
basics.dtypes

,0
tconst,object
titleType,object
primaryTitle,object
originalTitle,object
isAdult,float64
startYear,float64
endYear,float64
runtimeMinutes,object
genres,object


Convert `float64` into `int64`

In [ ]:
basics["isAdult"] = pd.to_numeric(basics["isAdult"], errors="coerce").astype("Int64")
basics["startYear"] = pd.to_numeric(basics["startYear"], errors="coerce").astype("Int64")
basics["endYear"] = pd.to_numeric(basics["endYear"], errors="coerce").astype("Int64")
basics.dtypes

,0
tconst,object
titleType,object
primaryTitle,object
originalTitle,object
isAdult,Int64
startYear,Int64
endYear,Int64
runtimeMinutes,object
genres,object


Column `runtimeMinutes` should be type `Int64`

In [ ]:
basics['runtimeMinutes'].unique()

array(['1', '5', '12', '45', '2', nan, '3', '100', '13', '6', '4', '40',
       '11', '9', '10', '15', '21', '16', '8', '14', '7', '24', '17',
       '70', '90', '20', '25', '120', '36', '18', '30', '19', '44', '38',
       '58', '22', '33', '50', '34', '28', '35', '42', '23', '51', '52',
       '60', '92', '56', '26', '53', '31', '41', '48', '47', '68', '29',
       '71', '43', '46', '27', '32', '76', '113', '55', '150', '300',
       '121', '89', '37', '69', '54', '64', '96', '49', '61', '39', '101',
       '57', '99', '110', '85', '88', '86', '170', '63', '72', '78', '75',
       '450', '148', '80', '124', '220', '59', '112', '1428', '84', '65',
       '74', '105', '310', '73', '82', '81', '199', '139', '109', '67',
       '219', '195', '440', '77', '293', '97', '79', '62', '66', '250',
       '421', '360', '180', '93', '163', '400', '95', '165', '83', '116',
       '320', '125', '127', '138', '460', '350', '374', '480', '330',
       '87', '130', '108', '600', '240', '117', '410', 

Looks like there are text in `runtimeMinutes` that should belong to `genres`

In [ ]:
misplaced_genres = ['Reality-TV', 'Talk-Show', 'Documentary','Family,Game-Show','Animation,Comedy,Family','News,Talk-Show','Comedy,News,Talk-Show','Documentary,Reality-TV','Comedy,Drama,Fantasy','Fantasy,Horror,Mystery','Action,Fantasy,Horror', 'Action,Horror,Mystery','Comedy,Drama,Horror','Action,Adventure,Drama','Drama,Fantasy,Horror','Game-Show,Reality-TV']
basics_misplaced_genres = basics[basics['runtimeMinutes'].isin(misplaced_genres)]
basics_misplaced_genres

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
1096717,tt10233364,tvEpisode,Rolling in the Deep Dish,Rolling in the Deep Dish,0,2019,<NA>,Reality-TV,NaN
1506173,tt10970874,tvEpisode,Die Bauhaus-Stadt Tel Aviv - Vorbild für die Metropolen der Moderne?,Die Bauhaus-Stadt Tel Aviv - Vorbild für die Metropolen der Moderne?,0,2019,<NA>,Talk-Show,NaN
1893321,tt11670006,tvEpisode,...ein angenehmer Unbequemer...,...ein angenehmer Unbequemer...,0,1981,<NA>,Documentary,NaN
2004106,tt11868642,tvEpisode,GGN Heavyweight Championship Lungs With Mike Tyson and Snoop,GGN Heavyweight Championship Lungs With Mike Tyson and Snoop,0,2020,<NA>,Talk-Show,NaN
2157353,tt12149332,tvEpisode,Jeopardy! College Championship Semifinal Game 3,Jeopardy! College Championship Semifinal Game 3,0,2020,<NA>,"Family,Game-Show",NaN
...,...,...,...,...,...,...,...,...,...
8525558,tt35473846,tvEpisode,X '1963,X '1963,0,1985,<NA>,"Action,Fantasy,Horror",NaN
8525574,tt35473863,tvEpisode,Reptilicus '1961,Reptilicus '1961,0,1985,<NA>,"Action,Fantasy,Horror",NaN
8717074,tt3984412,tvEpisode,"I'm Not Going to Come Last, I'm Just Going to Die on The Amazing Race","I'm Not Going to Come Last, I'm Just Going to Die on The Amazing Race",0,2014,<NA>,"Game-Show,Reality-TV",NaN
11346028,tt9822816,tvEpisode,Zwischen Vertuschung und Aufklärung - Missbrauchsgipfel im Vatikan,Zwischen Vertuschung und Aufklärung - Missbrauchsgipfel im Vatikan,0,2019,<NA>,Talk-Show,NaN


Move these values to the correct field, leave `runtimeMinutes` as `nan`

In [ ]:
basics_misplaced_genres['genres'] = basics_misplaced_genres['runtimeMinutes']
basics_misplaced_genres['runtimeMinutes'] = float('nan')
basics.loc[basics_misplaced_genres.index] = basics_misplaced_genres

<ipython-input-33-57f50934d59d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  basics_misplaced_genres['genres'] = basics_misplaced_genres['runtimeMinutes']
<ipython-input-33-57f50934d59d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  basics_misplaced_genres['runtimeMinutes'] = float('nan')


Cast `runtimeMinutes` to `Int64`

In [ ]:
basics["runtimeMinutes"] = pd.to_numeric(basics["runtimeMinutes"], errors="coerce").astype("Int64")

In [ ]:
basics.dtypes

,0
tconst,object
titleType,object
primaryTitle,object
originalTitle,object
isAdult,Int64
startYear,Int64
endYear,Int64
runtimeMinutes,Int64
genres,object


Final missing values and column type check

In [ ]:
basics.isna().sum()

,0
tconst,0
titleType,0
primaryTitle,0
originalTitle,0
isAdult,0
startYear,1419781
endYear,10887267
runtimeMinutes,7552951
genres,499327


### Data Cleaning for `episode` Dataset



In [ ]:
episode.dtypes

,0
tconst,object
parentTconst,object
seasonNumber,float64
episodeNumber,float64


In [ ]:
episode.isna().sum()

,0
tconst,0
parentTconst,0
seasonNumber,1776157
episodeNumber,1776157


Remove all rows with missing `seasonNumber` or `episodeNumber`

In [ ]:
episode.dropna(inplace=True)
episode.isna().sum()

,0
tconst,0
parentTconst,0
seasonNumber,0
episodeNumber,0


In [ ]:
episode['seasonNumber'] = episode['seasonNumber'].astype(int)
episode['episodeNumber'] = episode['episodeNumber'].astype(int)
episode.dtypes

,0
tconst,object
parentTconst,object
seasonNumber,int64
episodeNumber,int64


### Data Cleaning for `ratings` Dataset

In [ ]:
ratings.dtypes

,0
tconst,object
averageRating,float64
numVotes,int64


In [ ]:
ratings.isna().sum()

,0
tconst,0
averageRating,0
numVotes,0


## Data Export for SquerryDB GraphQL

In [ ]:
# Merge basics and ratings
all_ratings = pd.merge(basics, ratings, on='tconst', how='inner')
print(all_ratings.shape)

# Filter episode DataFrame
valid_tconsts = set(all_ratings['tconst'])
all_episodes = episode[
    episode['tconst'].isin(valid_tconsts) & episode['parentTconst'].isin(valid_tconsts)
]
print(all_episodes.shape, all_episodes.tconst.nunique(), all_episodes.parentTconst.nunique())

# Filter basics_ratings_merged DataFrame
all_ratings = all_ratings[
    all_ratings['tconst'].isin(all_episodes['tconst']) | all_ratings['tconst'].isin(all_episodes['parentTconst'])
]
print(all_ratings.shape)

(1500755, 11)
(753424, 4) 753424 39225
(792649, 11)


In [ ]:
# prompt: in all_episodes, create all_series from all_episodes that only have rows with tconst in episode['parentTconst']. create all_episodes from all_episodes merge with all_ratings on all_episodes.tconst

# Create all_series from all_episodes
all_series = all_ratings[all_ratings['tconst'].isin(all_episodes['parentTconst'])]
print(all_series.shape)

# Create all_episodes from all_episodes merged with all_ratings
all_episodes = pd.merge(all_episodes, all_ratings, on='tconst', how='inner')
print(all_episodes.shape)

(39225, 11)
(753424, 14)


In [ ]:
all_series.head(3)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
22082,tt0039123,tvSeries,Kraft Theatre,Kraft Television Theatre,0,1947,1958,60,Drama,7.9,233
22083,tt0039125,tvSeries,Public Prosecutor,Public Prosecutor,0,1947,1951,20,"Crime, Drama, Mystery",6.0,35
22844,tt0040021,tvSeries,Actor's Studio,Actor's Studio,0,1948,1950,30,Drama,7.1,94


In [ ]:
all_series.dtypes

,0
tconst,object
titleType,object
primaryTitle,object
originalTitle,object
isAdult,Int64
startYear,Int64
endYear,Int64
runtimeMinutes,Int64
genres,object
averageRating,float64


In [ ]:
all_series.isna().sum()

,0
tconst,0
titleType,0
primaryTitle,0
originalTitle,0
isAdult,0
startYear,4
endYear,14388
runtimeMinutes,12786
genres,228
averageRating,0


In [ ]:
all_episodes.head(2)

,tconst,parentTconst,seasonNumber,episodeNumber,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0041951,tt0041038,1,9,tvEpisode,The Tenderfeet,The Tenderfeet,0,1949,<NA>,30,Western,7.6,99
1,tt0042816,tt0989125,1,17,tvEpisode,Othello,Othello,0,1950,<NA>,143,Drama,7.6,12


In [ ]:
all_episodes.dtypes

,0
tconst,object
parentTconst,object
seasonNumber,int64
episodeNumber,int64
titleType,object
primaryTitle,object
originalTitle,object
isAdult,Int64
startYear,Int64
endYear,Int64


In [ ]:
all_episodes.isna().sum()

,0
tconst,0
parentTconst,0
seasonNumber,0
episodeNumber,0
titleType,0
primaryTitle,0
originalTitle,0
isAdult,0
startYear,149
endYear,753424


In [ ]:
all_episodes.to_csv('episodes.csv', index=False)
all_series.to_csv('series.csv', index=False)